In [1]:
pip install MetaTrader5 numpy pandas matplotlib scipy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import MetaTrader5 as mt5
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Arc
import time
from datetime import datetime
import math
from scipy.optimize import curve_fit
import threading
# import mplfinance as mpf

# Configuration parameters
class Config:
    symbol = "EURUSD.m"            # Trading symbol
    timeframe = mt5.TIMEFRAME_H1  # Timeframe
    arc_lookback = 100           # Number of candles to analyze for arcs
    risk_percent = 1             # Risk percentage per trade (1%)
    profit_ratio = 3             # Risk:Reward ratio (3:1)
    lots = 0.1                   # Default lot size
    max_arcs = 5                 # Maximum number of arcs to fit and display
    momentum_decay_factor = 0.5  # Half-life decay factor for momentum
    stop_loss_multiplier = 1.5   # SL multiplier relative to arc volatility
    visual_update_seconds = 10   # Update visualization every X seconds

class TradingBot:
    def __init__(self):
        self.config = Config()
        self.running = False
        self.trades = []
        self.current_arcs = []
        self.decision_points = []
        self.fig = None
        self.ax = None
        self.visual_thread = None
        
    def initialize_mt5(self):
        """Initialize connection to MetaTrader 5"""
        if not mt5.initialize():
            print("MT5 initialization failed")
            return False
            
        print(f"MT5 connected: {mt5.terminal_info()}")
        print(f"MT5 version: {mt5.version()}")
        return True
    
    def get_account_info(self):
        """Get account information"""
        account_info = mt5.account_info()
        if account_info is None:
            print("Failed to get account info")
            return None
        
        return {
            "balance": account_info.balance,
            "equity": account_info.equity,
            "profit": account_info.profit,
            "margin": account_info.margin,
            "free_margin": account_info.margin_free,
        }
    
    def get_market_data(self, count=None):
        """Get market data for the configured symbol and timeframe"""
        if count is None:
            count = self.config.arc_lookback * 2
            
        rates = mt5.copy_rates_from_pos(self.config.symbol, self.config.timeframe, 0, count)
        if rates is None:
            print("Failed to get market data")
            return None
            
        df = pd.DataFrame(rates)
        df['time'] = pd.to_datetime(df['time'], unit='s')
        df.set_index('time', inplace=True)
        return df
    
    def fit_arc(self, data, segment_start, segment_end):
        """Fit an arc to price data between start and end indices"""
        x = np.arange(segment_end - segment_start)
        y = data.iloc[segment_start:segment_end]['close'].values
        
        # Normalize data for better fitting
        x_norm = x / np.max(x)
        y_norm = y / np.max(y)
        
        try:
            # Fit a quadratic function (Arc approximation)
            def arc_function(x, a, b, c):
                return a * x**2 + b * x + c
                
            params, _ = curve_fit(arc_function, x_norm, y_norm)
            
            # Calculate the arc properties
            a, b, c = params
            
            # Center of the arc (vertex of the parabola)
            center_x = -b / (2 * a)
            center_y = arc_function(center_x, a, b, c)
            
            # Direction of the arc (concave up or down)
            direction = 1 if a > 0 else -1
            
            # Convert back to original scale
            center_x = center_x * np.max(x) + segment_start
            center_y = center_y * np.max(y)
            
            # Calculate the radius (approximate)
            r = 1 / (2 * abs(a))
            r = r * np.max(x)
            
            return {
                "center_x": center_x,
                "center_y": center_y,
                "radius": r,
                "direction": direction,
                "start_idx": segment_start,
                "end_idx": segment_end,
                "params": params,
                "max_price": np.max(y),
                "min_price": np.min(y),
                "volatility": np.std(y)
            }
        except:
            # If fitting fails, return None
            return None
    
    def detect_arcs(self, data):
        """Detect mathematical arcs in the price data"""
        arcs = []
        
        # Define different segment sizes to detect arcs of varying sizes
        segment_sizes = [20, 30, 40, 50, 60]
        
        for size in segment_sizes:
            # Skip if size is larger than available data
            if size >= len(data):
                continue
                
            # Slide through the data with overlapping windows
            for i in range(0, len(data) - size, max(5, size // 4)):
                arc = self.fit_arc(data, i, i + size)
                if arc is not None:
                    # Calculate quality of arc fit based on volatility and direction consistency
                    price_range = arc["max_price"] - arc["min_price"]
                    quality = price_range / arc["volatility"] if arc["volatility"] > 0 else 0
                    arc["quality"] = quality
                    arcs.append(arc)
        
        # Sort arcs by quality and keep only the best ones
        arcs = sorted(arcs, key=lambda x: x["quality"], reverse=True)
        return arcs[:self.config.max_arcs]
        
    def find_tangent_intersections(self, arcs, data):
        """Find tangent intersections between arcs"""
        intersections = []
        
        for i in range(len(arcs)):
            for j in range(i+1, len(arcs)):
                arc1 = arcs[i]
                arc2 = arcs[j]
                
                # Calculate potential intersection points between arcs
                dx = arc2["center_x"] - arc1["center_x"]
                dy = arc2["center_y"] - arc1["center_y"]
                dist = math.sqrt(dx**2 + dy**2)
                
                # Skip if arcs are too far apart
                if dist > arc1["radius"] + arc2["radius"]:
                    continue
                
                # Check if arcs have opposing directions (one up, one down)
                if arc1["direction"] != arc2["direction"]:
                    # Find the index near the potential intersection
                    mid_x = (arc1["center_x"] + arc2["center_x"]) / 2
                    idx = int(mid_x)
                    idx = max(0, min(idx, len(data) - 1))
                    
                    price = data.iloc[idx]['close']
                    timestamp = data.index[idx]
                    
                    # Calculate momentum decay
                    time_factor = 1 - min(1, (len(data) - 1 - idx) / len(data))
                    momentum = math.exp(-self.config.momentum_decay_factor * time_factor)
                    
                    intersections.append({
                        "idx": idx,
                        "price": price,
                        "timestamp": timestamp,
                        "arc1": arc1,
                        "arc2": arc2,
                        "type": "buy" if arc1["direction"] > 0 and arc2["direction"] < 0 else "sell",
                        "momentum": momentum,
                        "strength": (arc1["quality"] + arc2["quality"]) / 2 * momentum
                    })
        
        # Sort by strength
        intersections = sorted(intersections, key=lambda x: x["strength"], reverse=True)
        return intersections
    
    def calculate_position_size(self, stop_loss_pips):
        """Calculate position size based on risk percentage"""
        account_info = self.get_account_info()
        if account_info is None:
            return self.config.lots
            
        # Get symbol info for pip value calculation
        symbol_info = mt5.symbol_info(self.config.symbol)
        if symbol_info is None:
            return self.config.lots
            
        # Calculate pip value
        pip_value = symbol_info.trade_tick_value * (symbol_info.point / symbol_info.trade_tick_size)
        
        # Calculate risk amount
        risk_amount = account_info["balance"] * (self.config.risk_percent / 100)
        
        # Calculate position size
        position_size = risk_amount / (stop_loss_pips * pip_value)
        
        # Round to standard lot size
        position_size = round(position_size / 0.01) * 0.01
        
        # Minimum of 0.01 lots
        position_size = max(0.01, position_size)
        
        return position_size
    
    def place_order(self, order_type, price, stop_loss, take_profit):
        """Place an order with MT5"""
        symbol_info = mt5.symbol_info(self.config.symbol)
        if symbol_info is None:
            print(f"Failed to get symbol info for {self.config.symbol}")
            return None
            
        # Calculate position size based on risk
        pip_size = symbol_info.point * 10
        stop_loss_pips = abs(price - stop_loss) / pip_size
        lots = self.calculate_position_size(stop_loss_pips)
        
        print(f"Placing {order_type} order: Price={price}, SL={stop_loss}, TP={take_profit}, Lots={lots}")
        
        # Prepare order request
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": self.config.symbol,
            "volume": lots,
            "type": mt5.ORDER_TYPE_BUY if order_type == "buy" else mt5.ORDER_TYPE_SELL,
            "price": price,
            "sl": stop_loss,
            "tp": take_profit,
            "deviation": 20,  # Allow price slippage up to 20 points
            "magic": 123456,  # Magic number for identifying bot orders
            "comment": "Arc Trading Bot",
            "type_time": mt5.ORDER_TIME_GTC,  # Good till canceled
            "type_filling": mt5.ORDER_FILLING_IOC,  # Immediate or cancel
        }
        
        # Send order
        result = mt5.order_send(request)
        if result.retcode != mt5.TRADE_RETCODE_DONE:
            print(f"Order failed: {result.retcode}, {result.comment}")
            return None
            
        print(f"Order placed successfully. Ticket: {result.order}")
        return result.order
    
    def check_signals(self, data):
        """Check for trading signals based on arc intersections"""
        arcs = self.detect_arcs(data)
        self.current_arcs = arcs  # Store for visualization
        
        intersections = self.find_tangent_intersections(arcs, data)
        self.decision_points = intersections  # Store for visualization
        
        # Get the latest price
        current_price = data.iloc[-1]['close']
        
        # Filter recent intersections near the current price
        recent_signals = [
            x for x in intersections 
            if x["idx"] > len(data) - 20 and  # Within last 20 candles
            abs(x["price"] - current_price) / current_price < 0.01  # Within 1% of current price
        ]
        
        if not recent_signals:
            return None
            
        # Get the strongest signal
        signal = recent_signals[0]
        
        # Calculate stop loss and take profit
        volatility = max(signal["arc1"]["volatility"], signal["arc2"]["volatility"])
        sl_distance = volatility * self.config.stop_loss_multiplier
        tp_distance = sl_distance * self.config.profit_ratio
        
        if signal["type"] == "buy":
            stop_loss = current_price - sl_distance
            take_profit = current_price + tp_distance
            return {
                "type": "buy",
                "price": current_price,
                "stop_loss": stop_loss,
                "take_profit": take_profit,
                "strength": signal["strength"],
                "intersection": signal
            }
        else:
            stop_loss = current_price + sl_distance
            take_profit = current_price - tp_distance
            return {
                "type": "sell",
                "price": current_price,
                "stop_loss": stop_loss,
                "take_profit": take_profit,
                "strength": signal["strength"],
                "intersection": signal
            }
    
    def visualize_strategy(self, data):
        """Visualize the trading strategy with arcs and decision points"""
        # Create a plot with two subplots (price chart and metrics)
        if self.fig is None:
            plt.ion()  # Enable interactive mode
            self.fig, (self.ax, self.ax2) = plt.subplots(2, 1, figsize=(12, 8), gridspec_kw={'height_ratios': [3, 1]})
            plt.subplots_adjust(hspace=0.3)
        
        # Clear previous plots
        self.ax.clear()
        self.ax2.clear()
        
        # Plot price chart
        self.ax.plot(data.index, data['close'], label='Close Price', color='blue')
        
        # Plot arcs
        for arc in self.current_arcs:
            center_x = data.index[int(min(arc["center_x"], len(data) - 1))]
            center_y = arc["center_y"]
            radius = arc["radius"] * 10  # Scale for visualization
            
            # Create arc patch
            start_angle = 0
            end_angle = 180
            
            if arc["direction"] < 0:  # Concave down
                start_angle = 180
                end_angle = 360
                
            arc_patch = Arc(
                (center_x, center_y),
                width=radius,
                height=radius/4,  # Flatten the arc for better visualization
                angle=0,
                theta1=start_angle,
                theta2=end_angle,
                color='green' if arc["direction"] > 0 else 'red',
                alpha=0.7,
                lw=2
            )
            self.ax.add_patch(arc_patch)
        
        # Plot decision points
        for point in self.decision_points:
            idx = min(point["idx"], len(data) - 1)
            x = data.index[idx]
            y = point["price"]
            
            marker = '^' if point["type"] == "buy" else 'v'
            color = 'green' if point["type"] == "buy" else 'red'
            
            self.ax.scatter(x, y, marker=marker, color=color, s=100, alpha=0.8)
            
            # Draw tangent lines
            if idx > 0 and idx < len(data) - 1:
                self.ax.plot(
                    [data.index[idx-1], data.index[idx+1]],
                    [data.iloc[idx-1]['close'], data.iloc[idx+1]['close']],
                    '--', color=color, alpha=0.6
                )
                
        # Plot active trades
        for trade in self.trades:
            if trade["active"]:
                self.ax.axhline(y=trade["entry_price"], color='black', linestyle='-', alpha=0.5)
                self.ax.axhline(y=trade["stop_loss"], color='red', linestyle='--', alpha=0.5)
                self.ax.axhline(y=trade["take_profit"], color='green', linestyle='--', alpha=0.5)
        
        # Plot momentum decay
        momentum_values = []
        for i in range(len(data)):
            time_factor = 1 - min(1, (len(data) - 1 - i) / len(data))
            momentum = math.exp(-self.config.momentum_decay_factor * time_factor)
            momentum_values.append(momentum)
            
        self.ax2.plot(data.index, momentum_values, label='Wave Momentum', color='purple')
        self.ax2.set_title('Market Momentum (Half-life Decay)')
        self.ax2.set_ylim(0, 1.1)
        self.ax2.legend()
        
        # Set titles and labels
        self.ax.set_title(f'Arc Trading Strategy - {self.config.symbol}')
        self.ax.set_ylabel('Price')
        self.ax.legend()
        
        # Format x-axis to show dates clearly
        self.fig.autofmt_xdate()
        
        # Redraw
        self.fig.canvas.draw()
        self.fig.canvas.flush_events()
        
    def update_trades(self):
        """Update status of active trades"""
        active_trades = [t for t in self.trades if t["active"]]
        
        for trade in active_trades:
            # Get current price
            tick = mt5.symbol_info_tick(self.config.symbol)
            if tick is None:
                continue
                
            current_price = tick.bid if trade["type"] == "buy" else tick.ask
            
            # Check if stop loss hit
            if (trade["type"] == "buy" and current_price <= trade["stop_loss"]) or \
               (trade["type"] == "sell" and current_price >= trade["stop_loss"]):
                trade["active"] = False
                trade["exit_price"] = trade["stop_loss"]
                trade["exit_time"] = datetime.now()
                trade["profit"] = trade["stop_loss"] - trade["entry_price"] if trade["type"] == "buy" else trade["entry_price"] - trade["stop_loss"]
                print(f"Stop loss hit for trade {trade['id']}. Loss: {trade['profit']}")
                
            # Check if take profit hit
            elif (trade["type"] == "buy" and current_price >= trade["take_profit"]) or \
                 (trade["type"] == "sell" and current_price <= trade["take_profit"]):
                trade["active"] = False
                trade["exit_price"] = trade["take_profit"]
                trade["exit_time"] = datetime.now()
                trade["profit"] = trade["take_profit"] - trade["entry_price"] if trade["type"] == "buy" else trade["entry_price"] - trade["take_profit"]
                print(f"Take profit hit for trade {trade['id']}. Profit: {trade['profit']}")
    
    def visualization_thread(self):
        """Thread to update visualization regularly"""
        while self.running:
            try:
                data = self.get_market_data()
                if data is not None:
                    self.visualize_strategy(data)
            except Exception as e:
                print(f"Visualization error: {e}")
            
            time.sleep(self.config.visual_update_seconds)
    
    def run(self):
        """Main trading loop"""
        if not self.initialize_mt5():
            print("Failed to initialize MT5. Exiting.")
            return
        
        self.running = True
        
        # Start visualization thread
        self.visual_thread = threading.Thread(target=self.visualization_thread)
        self.visual_thread.daemon = True
        self.visual_thread.start()
        
        try:
            print(f"Starting trading bot for {self.config.symbol}...")
            
            while self.running:
                try:
                    # Get market data
                    data = self.get_market_data()
                    if data is None:
                        time.sleep(10)
                        continue
                        
                    # Update existing trades
                    self.update_trades()
                    
                    # Check for signals
                    signal = self.check_signals(data)
                    
                    # Execute trade if signal is strong enough
                    if signal and signal["strength"] > 0.6:  # Threshold for signal strength
                        ticket = self.place_order(
                            signal["type"], 
                            signal["price"], 
                            signal["stop_loss"], 
                            signal["take_profit"]
                        )
                        
                        if ticket:
                            # Record trade
                            trade = {
                                "id": len(self.trades) + 1,
                                "ticket": ticket,
                                "type": signal["type"],
                                "entry_price": signal["price"],
                                "stop_loss": signal["stop_loss"],
                                "take_profit": signal["take_profit"],
                                "entry_time": datetime.now(),
                                "active": True,
                                "exit_price": None,
                                "exit_time": None,
                                "profit": None
                            }
                            self.trades.append(trade)
                            print(f"New trade #{trade['id']} opened: {signal['type']} at {signal['price']}")
                            
                    # Sleep to avoid excessive API calls
                    time.sleep(30)
                    
                except Exception as e:
                    print(f"Error in trading loop: {e}")
                    time.sleep(60)  # Longer sleep on error
                    
        finally:
            self.running = False
            if self.visual_thread and self.visual_thread.is_alive():
                self.visual_thread.join(timeout=1)
            
            # Clean up resources
            plt.close('all')
            mt5.shutdown()

# Main entry point
if __name__ == "__main__":
    bot = TradingBot()
    bot.run()

MT5 connected: TerminalInfo(community_account=False, community_connection=False, connected=True, dlls_allowed=True, trade_allowed=False, tradeapi_disabled=False, email_enabled=False, ftp_enabled=False, notifications_enabled=False, mqid=True, build=4755, maxbars=100000, codepage=0, ping_last=217454, community_balance=0.0, retransmission=3.8797689618483053, company='MetaQuotes Software Corp.', name='MetaTrader 5', language='English', path='C:\\Program Files\\MetaTrader 5', data_path='C:\\Users\\Jelius\\AppData\\Roaming\\MetaQuotes\\Terminal\\D0E8209F77C8CF37AD8BF550E51FF075', commondata_path='C:\\Users\\Jelius\\AppData\\Roaming\\MetaQuotes\\Terminal\\Common')
MT5 version: (500, 4755, '13 Dec 2024')
Failed to get market data
Starting trading bot for EURUSD...
Failed to get market data
Failed to get market data
Failed to get market data
Failed to get market data
Failed to get market data
Failed to get market dataFailed to get market data

Failed to get market data
Failed to get market data

KeyboardInterrupt: 

In [6]:
import MetaTrader5 as mt5
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Arc
import time
from datetime import datetime
import math
from scipy.optimize import curve_fit
import threading
# import mplfinance as mpf

# Configuration parameters
class Config:
    symbol = "EURUSD.m"            # Trading symbol
    timeframe = mt5.TIMEFRAME_H1  # Timeframe
    arc_lookback = 100           # Number of candles to analyze for arcs
    risk_percent = 1             # Risk percentage per trade (1%)
    profit_ratio = 3             # Risk:Reward ratio (3:1)
    lots = 0.1                   # Default lot size
    max_arcs = 5                 # Maximum number of arcs to fit and display
    momentum_decay_factor = 0.5  # Half-life decay factor for momentum
    stop_loss_multiplier = 1.5   # SL multiplier relative to arc volatility
    visual_update_seconds = 10   # Update visualization every X seconds
    alternative_symbols = ["GBPUSD", "USDJPY", "AUDUSD", "USDCAD"]  # Alternative symbols to try if main symbol fails
    fallback_to_synthetic = True # Use synthetic data if real data cannot be obtained

class TradingBot:
    def __init__(self):
        self.config = Config()
        self.running = False
        self.trades = []
        self.current_arcs = []
        self.decision_points = []
        self.fig = None
        self.ax = None
        self.visual_thread = None
        
    def initialize_mt5(self):
        """Initialize connection to MetaTrader 5 with enhanced error handling"""
        try:
            # Check if already initialized
            if mt5.terminal_info() is not None:
                print("MT5 already initialized")
                return True
                
            # Attempt to initialize
            if not mt5.initialize():
                error_code = mt5.last_error()
                print(f"MT5 initialization failed with error code: {error_code}")
                
                # Provide specific troubleshooting advice based on error code
                if error_code == 10000:
                    print("No error returned - check if MT5 is running")
                elif error_code == 10013:
                    print("MT5 initialization failed - Invalid protocol version")
                elif error_code == 10018:
                    print("MT5 DLL not found or incompatible - reinstall MT5")
                elif error_code == 10021:
                    print("MT5 terminal is not connected to the server - check your internet connection")
                else:
                    print("Check that MT5 is running and that the Python bitness matches MT5 bitness (both 32-bit or both 64-bit)")
                
                return False
            
            # Get terminal info and version
            terminal_info = mt5.terminal_info()
            if terminal_info is None:
                print("Failed to get terminal info")
                return False
                
            version_info = mt5.version()
            if version_info is None:
                print("Failed to get version info")
                return False
            
            print(f"MT5 connected successfully:")
            print(f"- Terminal path: {terminal_info.path}")
            print(f"- Trade allowed: {'Yes' if terminal_info.trade_allowed else 'No'}")
            print(f"- MT5 version: {version_info[0]}.{version_info[1]}")
            
            # Check if trading is allowed
            if not terminal_info.trade_allowed:
                print("Warning: Trading is not allowed in the terminal. Algorithmic trading may not work properly.")
            
            # Get account info
            account_info = mt5.account_info()
            if account_info is not None:
                print(f"- Account: {account_info.login} ({account_info.server})")
                print(f"- Balance: {account_info.balance} {account_info.currency}")
            else:
                print("Warning: Could not get account information")
            
            return True
            
        except Exception as e:
            print(f"Error during MT5 initialization: {e}")
            return False
    
    def get_account_info(self):
        """Get account information"""
        account_info = mt5.account_info()
        if account_info is None:
            print("Failed to get account info")
            return None
        
        return {
            "balance": account_info.balance,
            "equity": account_info.equity,
            "profit": account_info.profit,
            "margin": account_info.margin,
            "free_margin": account_info.margin_free,
        }
    
    def get_market_data(self, count=None):
        """Get market data for the configured symbol and timeframe with improved error handling"""
        if count is None:
            count = self.config.arc_lookback * 2
        
        # First check if symbol exists
        symbol_info = mt5.symbol_info(self.config.symbol)
        if symbol_info is None:
            print(f"Symbol {self.config.symbol} not found. Please check if the symbol name is correct.")
            # Try to get available symbols
            symbols = mt5.symbols_get()
            if symbols:
                print(f"Available symbols (showing first 10): {[s.name for s in symbols[:10]]}")
            return None
        
        # Ensure symbol is selected in Market Watch
        if not symbol_info.visible:
            print(f"Symbol {self.config.symbol} is not visible in Market Watch, trying to select it...")
            if not mt5.symbol_select(self.config.symbol, True):
                print(f"Failed to select {self.config.symbol}")
                return None
        
        # Try different methods to get data
        try:
            # Method 1: copy_rates_from_pos
            rates = mt5.copy_rates_from_pos(self.config.symbol, self.config.timeframe, 0, count)
            if rates is not None and len(rates) > 0:
                df = pd.DataFrame(rates)
                df['time'] = pd.to_datetime(df['time'], unit='s')
                df.set_index('time', inplace=True)
                print(f"Successfully retrieved {len(df)} candles using copy_rates_from_pos")
                return df
            
            # Method 2: copy_rates_range (try last 30 days)
            from_date = datetime.now().timestamp() - (30 * 24 * 60 * 60)  # 30 days ago
            to_date = datetime.now().timestamp()
            rates = mt5.copy_rates_range(self.config.symbol, self.config.timeframe, 
                                        from_date, to_date)
            if rates is not None and len(rates) > 0:
                df = pd.DataFrame(rates)
                df['time'] = pd.to_datetime(df['time'], unit='s')
                df.set_index('time', inplace=True)
                print(f"Successfully retrieved {len(df)} candles using copy_rates_range")
                return df
            
            # Method 3: Try getting ticks and constructing OHLC (last resort)
            ticks = mt5.copy_ticks_from(self.config.symbol, datetime.now().timestamp() - (24 * 60 * 60), 100000, mt5.COPY_TICKS_ALL)
            if ticks is not None and len(ticks) > 0:
                print(f"Retrieved {len(ticks)} ticks, constructing OHLC data...")
                ticks_df = pd.DataFrame(ticks)
                ticks_df['time'] = pd.to_datetime(ticks_df['time'], unit='s')
                
                # Create 1-minute bars as a fallback
                ticks_df.set_index('time', inplace=True)
                ohlc = ticks_df['bid'].resample('1min').ohlc()
                ohlc.columns = ['open', 'high', 'low', 'close']
                
                # Add volume column
                volume = ticks_df.resample('1min').count().iloc[:, 0]
                ohlc['volume'] = volume
                
                print(f"Successfully constructed {len(ohlc)} OHLC bars from ticks")
                return ohlc.iloc[-count:] if len(ohlc) > count else ohlc
                
            print("Failed to get market data using all available methods")
            return None
            
        except Exception as e:
            print(f"Error getting market data: {e}")
            
            # As a last resort, generate synthetic data for testing
            if not hasattr(self, '_synthetic_data_warning_shown'):
                print("WARNING: Using synthetic data for testing purposes only!")
                self._synthetic_data_warning_shown = True
                
            # Generate synthetic data
            now = datetime.now()
            dates = pd.date_range(end=now, periods=count, freq='H')
            
            # Generate random walk prices
            np.random.seed(42)  # For reproducibility
            close_prices = 1.1 + np.cumsum(np.random.normal(0, 0.001, count))
            
            # Create synthetic DataFrame
            synthetic_df = pd.DataFrame({
                'open': close_prices * (1 + np.random.normal(0, 0.0002, count)),
                'high': close_prices * (1 + abs(np.random.normal(0, 0.0005, count))),
                'low': close_prices * (1 - abs(np.random.normal(0, 0.0005, count))),
                'close': close_prices,
                'volume': np.random.randint(100, 1000, count)
            }, index=dates)
            
            return synthetic_df
    
    def fit_arc(self, data, segment_start, segment_end):
        """Fit an arc to price data between start and end indices"""
        x = np.arange(segment_end - segment_start)
        y = data.iloc[segment_start:segment_end]['close'].values
        
        # Normalize data for better fitting
        x_norm = x / np.max(x)
        y_norm = y / np.max(y)
        
        try:
            # Fit a quadratic function (Arc approximation)
            def arc_function(x, a, b, c):
                return a * x**2 + b * x + c
                
            params, _ = curve_fit(arc_function, x_norm, y_norm)
            
            # Calculate the arc properties
            a, b, c = params
            
            # Center of the arc (vertex of the parabola)
            center_x = -b / (2 * a)
            center_y = arc_function(center_x, a, b, c)
            
            # Direction of the arc (concave up or down)
            direction = 1 if a > 0 else -1
            
            # Convert back to original scale
            center_x = center_x * np.max(x) + segment_start
            center_y = center_y * np.max(y)
            
            # Calculate the radius (approximate)
            r = 1 / (2 * abs(a))
            r = r * np.max(x)
            
            return {
                "center_x": center_x,
                "center_y": center_y,
                "radius": r,
                "direction": direction,
                "start_idx": segment_start,
                "end_idx": segment_end,
                "params": params,
                "max_price": np.max(y),
                "min_price": np.min(y),
                "volatility": np.std(y)
            }
        except:
            # If fitting fails, return None
            return None
    
    def detect_arcs(self, data):
        """Detect mathematical arcs in the price data"""
        arcs = []
        
        # Define different segment sizes to detect arcs of varying sizes
        segment_sizes = [20, 30, 40, 50, 60]
        
        for size in segment_sizes:
            # Skip if size is larger than available data
            if size >= len(data):
                continue
                
            # Slide through the data with overlapping windows
            for i in range(0, len(data) - size, max(5, size // 4)):
                arc = self.fit_arc(data, i, i + size)
                if arc is not None:
                    # Calculate quality of arc fit based on volatility and direction consistency
                    price_range = arc["max_price"] - arc["min_price"]
                    quality = price_range / arc["volatility"] if arc["volatility"] > 0 else 0
                    arc["quality"] = quality
                    arcs.append(arc)
        
        # Sort arcs by quality and keep only the best ones
        arcs = sorted(arcs, key=lambda x: x["quality"], reverse=True)
        return arcs[:self.config.max_arcs]
        
    def find_tangent_intersections(self, arcs, data):
        """Find tangent intersections between arcs"""
        intersections = []
        
        for i in range(len(arcs)):
            for j in range(i+1, len(arcs)):
                arc1 = arcs[i]
                arc2 = arcs[j]
                
                # Calculate potential intersection points between arcs
                dx = arc2["center_x"] - arc1["center_x"]
                dy = arc2["center_y"] - arc1["center_y"]
                dist = math.sqrt(dx**2 + dy**2)
                
                # Skip if arcs are too far apart
                if dist > arc1["radius"] + arc2["radius"]:
                    continue
                
                # Check if arcs have opposing directions (one up, one down)
                if arc1["direction"] != arc2["direction"]:
                    # Find the index near the potential intersection
                    mid_x = (arc1["center_x"] + arc2["center_x"]) / 2
                    idx = int(mid_x)
                    idx = max(0, min(idx, len(data) - 1))
                    
                    price = data.iloc[idx]['close']
                    timestamp = data.index[idx]
                    
                    # Calculate momentum decay
                    time_factor = 1 - min(1, (len(data) - 1 - idx) / len(data))
                    momentum = math.exp(-self.config.momentum_decay_factor * time_factor)
                    
                    intersections.append({
                        "idx": idx,
                        "price": price,
                        "timestamp": timestamp,
                        "arc1": arc1,
                        "arc2": arc2,
                        "type": "buy" if arc1["direction"] > 0 and arc2["direction"] < 0 else "sell",
                        "momentum": momentum,
                        "strength": (arc1["quality"] + arc2["quality"]) / 2 * momentum
                    })
        
        # Sort by strength
        intersections = sorted(intersections, key=lambda x: x["strength"], reverse=True)
        return intersections
    
    def calculate_position_size(self, stop_loss_pips):
        """Calculate position size based on risk percentage"""
        account_info = self.get_account_info()
        if account_info is None:
            return self.config.lots
            
        # Get symbol info for pip value calculation
        symbol_info = mt5.symbol_info(self.config.symbol)
        if symbol_info is None:
            return self.config.lots
            
        # Calculate pip value
        pip_value = symbol_info.trade_tick_value * (symbol_info.point / symbol_info.trade_tick_size)
        
        # Calculate risk amount
        risk_amount = account_info["balance"] * (self.config.risk_percent / 100)
        
        # Calculate position size
        position_size = risk_amount / (stop_loss_pips * pip_value)
        
        # Round to standard lot size
        position_size = round(position_size / 0.01) * 0.01
        
        # Minimum of 0.01 lots
        position_size = max(0.01, position_size)
        
        return position_size
    
    def place_order(self, order_type, price, stop_loss, take_profit):
        """Place an order with MT5"""
        symbol_info = mt5.symbol_info(self.config.symbol)
        if symbol_info is None:
            print(f"Failed to get symbol info for {self.config.symbol}")
            return None
            
        # Calculate position size based on risk
        pip_size = symbol_info.point * 10
        stop_loss_pips = abs(price - stop_loss) / pip_size
        lots = self.calculate_position_size(stop_loss_pips)
        
        print(f"Placing {order_type} order: Price={price}, SL={stop_loss}, TP={take_profit}, Lots={lots}")
        
        # Prepare order request
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": self.config.symbol,
            "volume": lots,
            "type": mt5.ORDER_TYPE_BUY if order_type == "buy" else mt5.ORDER_TYPE_SELL,
            "price": price,
            "sl": stop_loss,
            "tp": take_profit,
            "deviation": 20,  # Allow price slippage up to 20 points
            "magic": 123456,  # Magic number for identifying bot orders
            "comment": "Arc Trading Bot",
            "type_time": mt5.ORDER_TIME_GTC,  # Good till canceled
            "type_filling": mt5.ORDER_FILLING_IOC,  # Immediate or cancel
        }
        
        # Send order
        result = mt5.order_send(request)
        if result.retcode != mt5.TRADE_RETCODE_DONE:
            print(f"Order failed: {result.retcode}, {result.comment}")
            return None
            
        print(f"Order placed successfully. Ticket: {result.order}")
        return result.order
    
    def check_signals(self, data):
        """Check for trading signals based on arc intersections"""
        arcs = self.detect_arcs(data)
        self.current_arcs = arcs  # Store for visualization
        
        intersections = self.find_tangent_intersections(arcs, data)
        self.decision_points = intersections  # Store for visualization
        
        # Get the latest price
        current_price = data.iloc[-1]['close']
        
        # Filter recent intersections near the current price
        recent_signals = [
            x for x in intersections 
            if x["idx"] > len(data) - 20 and  # Within last 20 candles
            abs(x["price"] - current_price) / current_price < 0.01  # Within 1% of current price
        ]
        
        if not recent_signals:
            return None
            
        # Get the strongest signal
        signal = recent_signals[0]
        
        # Calculate stop loss and take profit
        volatility = max(signal["arc1"]["volatility"], signal["arc2"]["volatility"])
        sl_distance = volatility * self.config.stop_loss_multiplier
        tp_distance = sl_distance * self.config.profit_ratio
        
        if signal["type"] == "buy":
            stop_loss = current_price - sl_distance
            take_profit = current_price + tp_distance
            return {
                "type": "buy",
                "price": current_price,
                "stop_loss": stop_loss,
                "take_profit": take_profit,
                "strength": signal["strength"],
                "intersection": signal
            }
        else:
            stop_loss = current_price + sl_distance
            take_profit = current_price - tp_distance
            return {
                "type": "sell",
                "price": current_price,
                "stop_loss": stop_loss,
                "take_profit": take_profit,
                "strength": signal["strength"],
                "intersection": signal
            }
    
    def visualize_strategy(self, data):
        """Visualize the trading strategy with arcs and decision points"""
        # Create a plot with two subplots (price chart and metrics)
        if self.fig is None:
            plt.ion()  # Enable interactive mode
            self.fig, (self.ax, self.ax2) = plt.subplots(2, 1, figsize=(12, 8), gridspec_kw={'height_ratios': [3, 1]})
            plt.subplots_adjust(hspace=0.3)
        
        # Clear previous plots
        self.ax.clear()
        self.ax2.clear()
        
        # Plot price chart
        self.ax.plot(data.index, data['close'], label='Close Price', color='blue')
        
        # Plot arcs
        for arc in self.current_arcs:
            center_x = data.index[int(min(arc["center_x"], len(data) - 1))]
            center_y = arc["center_y"]
            radius = arc["radius"] * 10  # Scale for visualization
            
            # Create arc patch
            start_angle = 0
            end_angle = 180
            
            if arc["direction"] < 0:  # Concave down
                start_angle = 180
                end_angle = 360
                
            arc_patch = Arc(
                (center_x, center_y),
                width=radius,
                height=radius/4,  # Flatten the arc for better visualization
                angle=0,
                theta1=start_angle,
                theta2=end_angle,
                color='green' if arc["direction"] > 0 else 'red',
                alpha=0.7,
                lw=2
            )
            self.ax.add_patch(arc_patch)
        
        # Plot decision points
        for point in self.decision_points:
            idx = min(point["idx"], len(data) - 1)
            x = data.index[idx]
            y = point["price"]
            
            marker = '^' if point["type"] == "buy" else 'v'
            color = 'green' if point["type"] == "buy" else 'red'
            
            self.ax.scatter(x, y, marker=marker, color=color, s=100, alpha=0.8)
            
            # Draw tangent lines
            if idx > 0 and idx < len(data) - 1:
                self.ax.plot(
                    [data.index[idx-1], data.index[idx+1]],
                    [data.iloc[idx-1]['close'], data.iloc[idx+1]['close']],
                    '--', color=color, alpha=0.6
                )
                
        # Plot active trades
        for trade in self.trades:
            if trade["active"]:
                self.ax.axhline(y=trade["entry_price"], color='black', linestyle='-', alpha=0.5)
                self.ax.axhline(y=trade["stop_loss"], color='red', linestyle='--', alpha=0.5)
                self.ax.axhline(y=trade["take_profit"], color='green', linestyle='--', alpha=0.5)
        
        # Plot momentum decay
        momentum_values = []
        for i in range(len(data)):
            time_factor = 1 - min(1, (len(data) - 1 - i) / len(data))
            momentum = math.exp(-self.config.momentum_decay_factor * time_factor)
            momentum_values.append(momentum)
            
        self.ax2.plot(data.index, momentum_values, label='Wave Momentum', color='purple')
        self.ax2.set_title('Market Momentum (Half-life Decay)')
        self.ax2.set_ylim(0, 1.1)
        self.ax2.legend()
        
        # Set titles and labels
        self.ax.set_title(f'Arc Trading Strategy - {self.config.symbol}')
        self.ax.set_ylabel('Price')
        self.ax.legend()
        
        # Format x-axis to show dates clearly
        self.fig.autofmt_xdate()
        
        # Redraw
        self.fig.canvas.draw()
        self.fig.canvas.flush_events()
        
    def update_trades(self):
        """Update status of active trades"""
        active_trades = [t for t in self.trades if t["active"]]
        
        for trade in active_trades:
            # Get current price
            tick = mt5.symbol_info_tick(self.config.symbol)
            if tick is None:
                continue
                
            current_price = tick.bid if trade["type"] == "buy" else tick.ask
            
            # Check if stop loss hit
            if (trade["type"] == "buy" and current_price <= trade["stop_loss"]) or \
               (trade["type"] == "sell" and current_price >= trade["stop_loss"]):
                trade["active"] = False
                trade["exit_price"] = trade["stop_loss"]
                trade["exit_time"] = datetime.now()
                trade["profit"] = trade["stop_loss"] - trade["entry_price"] if trade["type"] == "buy" else trade["entry_price"] - trade["stop_loss"]
                print(f"Stop loss hit for trade {trade['id']}. Loss: {trade['profit']}")
                
            # Check if take profit hit
            elif (trade["type"] == "buy" and current_price >= trade["take_profit"]) or \
                 (trade["type"] == "sell" and current_price <= trade["take_profit"]):
                trade["active"] = False
                trade["exit_price"] = trade["take_profit"]
                trade["exit_time"] = datetime.now()
                trade["profit"] = trade["take_profit"] - trade["entry_price"] if trade["type"] == "buy" else trade["entry_price"] - trade["take_profit"]
                print(f"Take profit hit for trade {trade['id']}. Profit: {trade['profit']}")
    
    def visualization_thread(self):
        """Thread to update visualization regularly"""
        while self.running:
            try:
                data = self.get_market_data()
                if data is not None:
                    self.visualize_strategy(data)
            except Exception as e:
                print(f"Visualization error: {e}")
            
            time.sleep(self.config.visual_update_seconds)
    
    def run(self):
        """Main trading loop"""
        if not self.initialize_mt5():
            print("Failed to initialize MT5. Checking for alternatives...")
            
            if self.config.fallback_to_synthetic:
                print("Proceeding with synthetic data for demonstration purposes")
            else:
                print("Exiting.")
                return
        
        self.running = True
        
        # Test market data retrieval before starting
        print("Testing market data retrieval...")
        data = self.get_market_data(30)  # Get 30 candles for testing
        
        if data is None:
            print("Failed to get market data for the configured symbol.")
            
            # Try alternative symbols
            for alt_symbol in self.config.alternative_symbols:
                print(f"Trying alternative symbol: {alt_symbol}")
                original_symbol = self.config.symbol
                self.config.symbol = alt_symbol
                
                alt_data = self.get_market_data(30)
                if alt_data is not None:
                    print(f"Successfully retrieved data for {alt_symbol}, switching to this symbol")
                    break
                    
                # Restore original symbol
                self.config.symbol = original_symbol
            
            if data is None and not self.config.fallback_to_synthetic:
                print("Failed to get market data for any symbol. Exiting.")
                self.running = False
                return
        else:
            print(f"Successfully retrieved market data for {self.config.symbol}")
            print(f"Data sample: {data.head(3)}")
        
        # Start visualization thread
        self.visual_thread = threading.Thread(target=self.visualization_thread)
        self.visual_thread.daemon = True
        self.visual_thread.start()
        
        try:
            print(f"Starting trading bot for {self.config.symbol}...")
            
            while self.running:
                try:
                    # Get market data
                    data = self.get_market_data()
                    if data is None:
                        print("Failed to get market data, retrying in 10 seconds...")
                        time.sleep(10)
                        continue
                    
                    # Print some stats about the data
                    print(f"Working with {len(data)} candles from {data.index.min()} to {data.index.max()}")
                    
                    # Update existing trades
                    self.update_trades()
                    
                    # Check for signals
                    signal = self.check_signals(data)
                    
                    # Execute trade if signal is strong enough
                    if signal and signal["strength"] > 0.6:  # Threshold for signal strength
                        ticket = self.place_order(
                            signal["type"], 
                            signal["price"], 
                            signal["stop_loss"], 
                            signal["take_profit"]
                        )
                        
                        if ticket:
                            # Record trade
                            trade = {
                                "id": len(self.trades) + 1,
                                "ticket": ticket,
                                "type": signal["type"],
                                "entry_price": signal["price"],
                                "stop_loss": signal["stop_loss"],
                                "take_profit": signal["take_profit"],
                                "entry_time": datetime.now(),
                                "active": True,
                                "exit_price": None,
                                "exit_time": None,
                                "profit": None
                            }
                            self.trades.append(trade)
                            print(f"New trade #{trade['id']} opened: {signal['type']} at {signal['price']}")
                            
                    # Sleep to avoid excessive API calls
                    time.sleep(30)
                    
                except Exception as e:
                    print(f"Error in trading loop: {e}")
                    import traceback
                    traceback.print_exc()
                    time.sleep(60)  # Longer sleep on error
                    
        finally:
            self.running = False
            if self.visual_thread and self.visual_thread.is_alive():
                self.visual_thread.join(timeout=1)
            
            # Clean up resources
            plt.close('all')
            mt5.shutdown()

def main():
    """Main entry point with error handling"""
    try:
        print("Starting Mathematical Arc Trading Bot")
        print("=====================================")
        print("Checking MT5 availability...")
        
        # Check if MetaTrader5 module is available
        if 'MetaTrader5' not in sys.modules:
            print("MetaTrader5 module not found. Checking installation...")
            try:
                import pip
                installed_packages = [pkg.key for pkg in pip.get_installed_distributions()]
                if 'metaTrader5' not in installed_packages and 'MetaTrader5' not in installed_packages:
                    print("MetaTrader5 package not installed. Please install it using:")
                    print("pip install MetaTrader5")
                    
                    # Proceed with synthetic data mode
                    print("Continuing in SYNTHETIC DATA MODE (for demonstration only)")
                else:
                    print("MetaTrader5 is installed but not imported correctly. Check for import errors.")
            except:
                print("Could not check installed packages. Make sure MetaTrader5 is installed.")
        
        # Create and run the bot
        bot = TradingBot()
        bot.run()
        
    except KeyboardInterrupt:
        print("\nBot terminated by user.")
    except Exception as e:
        print(f"Critical error: {e}")
        import traceback
        traceback.print_exc()
        
    print("Bot shutdown complete.")

# Main entry point
if __name__ == "__main__":
    import sys
    main()

Starting Mathematical Arc Trading Bot
Checking MT5 availability...
MT5 connected successfully:
- Terminal path: C:\Program Files\MetaTrader 5
- Trade allowed: Yes
- MT5 version: 500.4755
- Account: 2001193758 (JustMarkets-Demo)
- Balance: 200.0 USD
Testing market data retrieval...
Successfully retrieved 30 candles using copy_rates_from_pos
Successfully retrieved market data for EURUSD.m
Data sample:                         open     high      low    close  tick_volume  spread  \
time                                                                           
2025-05-20 04:00:00  1.12275  1.12508  1.12275  1.12378         4161       8   
2025-05-20 05:00:00  1.12379  1.12458  1.12255  1.12316         2695       8   
2025-05-20 06:00:00  1.12316  1.12470  1.12289  1.12445         2848       8   

                     real_volume  
time                              
2025-05-20 04:00:00            0  
2025-05-20 05:00:00            0  
2025-05-20 06:00:00            0  
Starting trading bot 

In [ ]:
"""
Elliptical Arc Trading Bot with MT5 Integration
"""

import MetaTrader5 as mt5
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Arc
import math
from scipy.optimize import curve_fit
import threading
import time
from datetime import datetime
import sys

# Configuration Parameters
class Config:
    symbol = "EURUSD.m"
    timeframe = mt5.TIMEFRAME_H1
    arc_lookback = 100
    risk_percent = 1
    profit_ratio = 3
    lots = 0.1
    max_arcs = 5
    momentum_decay_factor = 0.5
    stop_loss_multiplier = 1.5
    visual_update_seconds = 10
    alternative_symbols = ["GBPUSD.m", "USDJPY.m", "AUDUSD.m", "USDCAD.m"]
    fallback_to_synthetic = True


class ArcTradingBot:
    def __init__(self):
        self.config = Config()
        self.running = False
        self.trades = []
        self.current_arcs = []
        self.decision_points = []
        self.fig = None
        self.ax = None
        self.ax2 = None
        self.visual_thread = None

    def initialize_mt5(self):
        if not mt5.initialize():
            print(f"MT5 initialization failed: {mt5.last_error()}")
            return False

        info = mt5.terminal_info()
        account = mt5.account_info()

        print("MT5 connected successfully:")
        print(f"- Path: {info.path}")
        print(f"- Trade Allowed: {'Yes' if info.trade_allowed else 'No'}")
        print(f"- Version: {mt5.version()[0]}")
        print(f"- Account: {account.login} ({account.server})")
        print(f"- Balance: {account.balance} {account.currency}")
        return True

    def get_market_data(self, count=None):
        if count is None:
            count = self.config.arc_lookback * 2

        symbol_info = mt5.symbol_info(self.config.symbol)
        if symbol_info is None:
            print(f"Symbol {self.config.symbol} not found.")
            return self.generate_synthetic(count)

        if not symbol_info.visible:
            mt5.symbol_select(self.config.symbol, True)

        rates = mt5.copy_rates_from_pos(self.config.symbol, self.config.timeframe, 0, count)
        if rates is not None:
            df = pd.DataFrame(rates)
            df['time'] = pd.to_datetime(df['time'], unit='s')
            df.set_index('time', inplace=True)
            return df

        print("Failed to retrieve market data. Using synthetic fallback...")
        return self.generate_synthetic(count)

    def generate_synthetic(self, count):
        now = datetime.now()
        dates = pd.date_range(end=now, periods=count, freq='H')
        prices = 1.1 + np.cumsum(np.random.normal(0, 0.001, count))
        return pd.DataFrame({
            'open': prices,
            'high': prices * (1 + np.abs(np.random.normal(0, 0.0005, count))),
            'low': prices * (1 - np.abs(np.random.normal(0, 0.0005, count))),
            'close': prices,
            'volume': np.random.randint(100, 1000, count)
        }, index=dates)

    def fit_arc(self, data, start, end):
        x = np.arange(end - start)
        y = data.iloc[start:end]['close'].values
        x_norm = x / len(x)
        y_mean = np.mean(y)
        y_std = np.std(y) or 1
        y_norm = (y - y_mean) / y_std

        def elliptical(x, a, b, c, d, e):
            return a * x**2 + b * x + c + d * np.sin(e * x)

        try:
            popt, _ = curve_fit(elliptical, x_norm, y_norm, p0=[0, 0, 0, 0.1, 5], maxfev=10000)
            fitted = elliptical(x_norm, *popt) * y_std + y_mean
            r_squared = 1 - sum((y - fitted)**2) / sum((y - y.mean())**2)
            curvature = abs(popt[3] * popt[4]) / (1 + abs(popt[0]))
            direction = 1 if fitted[-1] > fitted[0] else -1
            return {
                'start_idx': start,
                'end_idx': end,
                'fitted_values': fitted,
                'r_squared': r_squared,
                'curvature': curvature,
                'direction': direction,
                'params': popt,
                'volatility': np.std(y),
                'quality': r_squared * curvature,
                'size': end - start
            }
        except:
            return None

    def detect_arcs(self, data):
        arcs = []
        sizes = [20, 30, 40, 50, 60]
        for size in sizes:
            for i in range(0, len(data) - size, max(5, size // 4)):
                arc = self.fit_arc(data, i, i + size)
                if arc and arc["r_squared"] > 0.7 and arc["curvature"] > 0.05:
                    arcs.append(arc)
        return sorted(arcs, key=lambda a: a["quality"], reverse=True)

    def find_signals(self, arcs, data):
        signals = []
        for arc in arcs:
            start_idx = arc['start_idx']
            end_idx = arc['end_idx']
            segment_length = end_idx - start_idx
            y = data.iloc[start_idx:end_idx]['close'].values
            x = np.arange(segment_length)
            x_norm = x / segment_length
            a, b, c, d, e = arc['params']

            def elliptical_derivative(x_val, a_val, b_val, d_val, e_val):
                return 2 * a_val * x_val + b_val + d_val * e_val * np.cos(e_val * x_val)

            start_slope = elliptical_derivative(x_norm[0], a, b, d, e)
            end_slope = elliptical_derivative(x_norm[-1], a, b, d, e)
            angle_diff = math.degrees(math.atan(end_slope - start_slope))

            trending_up = angle_diff > 0
            idx = end_idx
            price = data.iloc[min(idx, len(data)-1)]['close']

            strength = min(1.0, abs(angle_diff)/90)
            strength *= arc['r_squared'] * arc['curvature']
            time_factor = 1 - min(1, (len(data)-1-idx)/len(data))
            momentum = math.exp(-self.config.momentum_decay_factor * time_factor)
            strength *= momentum

            if trending_up and angle_diff > 30:
                signals.append({'type': 'buy', 'price': price, 'idx': idx, 'strength': strength, 'volatility': arc['volatility']})
            elif not trending_up and angle_diff < -30:
                signals.append({'type': 'sell', 'price': price, 'idx': idx, 'strength': strength, 'volatility': arc['volatility']})

        return sorted(signals, key=lambda s: s['strength'], reverse=True)

    def calculate_position_size(self, sl_pips):
        account = mt5.account_info()
        if not account:
            print("⚠️ Failed to get account info, using default lots.")
            return self.config.lots

        symbol = mt5.symbol_info(self.config.symbol)
        if not symbol:
            print("⚠️ Failed to get symbol info.")
            return self.config.lots

        pip_value = symbol.trade_tick_value / symbol.point
        risk_amount = account.balance * (self.config.risk_percent / 100)
        lots = risk_amount / (sl_pips * pip_value)
        return round(max(lots, 0.01), 2)

    def place_order(self, order_type, price, sl, tp):
        symbol = mt5.symbol_info(self.config.symbol)
        if not symbol:
            print("⚠️ Failed to retrieve symbol info.")
            return None
    
        # 1) Calculate pip size & sl_pips as before
        pip_size = symbol.point * 10  # 5-digit broker
        sl_pips = abs(price - sl) / pip_size
        lots   = self.calculate_position_size(sl_pips)

        # 2) Enforce minimum stop distance
        min_dist = symbol.stops_level * symbol.point
        if order_type == "buy":
            # SL must be below price, TP above price
            sl = min(sl, price - min_dist)
            tp = max(tp, price + min_dist)
        else:
            # SL above price, TP below price
            sl = max(sl, price + min_dist)
            tp = min(tp, price - min_dist)
    
        # 3) Use FOK filling mode (the only one that works for you)
        filling_mode = mt5.ORDER_FILLING_FOK

        req = {
            "action":       mt5.TRADE_ACTION_DEAL,
            "symbol":       self.config.symbol,
            "volume":       lots,
            "type":         mt5.ORDER_TYPE_BUY   if order_type=="buy"  else mt5.ORDER_TYPE_SELL,
            "price":        price,
            "sl":           sl,
            "tp":           tp,
            "deviation":    20,
            "magic":        123456,
            "comment":      "Arc Trading Bot",
            "type_time":    mt5.ORDER_TIME_GTC,
            "type_filling": filling_mode,
        }

        res = mt5.order_send(req)
        if res.retcode != mt5.TRADE_RETCODE_DONE:
            print(f"❌ Order failed: {res.retcode} → {res.comment}")
            return None
    
        print(f"✅ Order placed: {order_type.upper()} @ {price:.5f}, SL={sl:.5f}, TP={tp:.5f}, Lots={lots}")
        return res.order


    


    def visualize(self, data):
        if not self.fig:
            plt.ion()
            self.fig, (self.ax, self.ax2) = plt.subplots(2, 1, figsize=(14, 8), gridspec_kw={'height_ratios': [3, 1]})
            plt.subplots_adjust(hspace=0.3)

        self.ax.clear()
        self.ax2.clear()
        self.ax.plot(data.index, data['close'], label="Price", color="blue")

        for arc in self.current_arcs[:self.config.max_arcs]:
            x_vals = [data.index[i] for i in range(arc['start_idx'], arc['end_idx'])]
            self.ax.plot(x_vals, arc['fitted_values'], color='green', linestyle='--')

        self.ax.set_title("Price with Arcs")
        self.ax.legend()
        self.ax.grid(True)

        plt.pause(0.01)

    def run(self):
        if not self.initialize_mt5():
            return

        self.running = True
        while self.running:
            data = self.get_market_data()
            self.current_arcs = self.detect_arcs(data)
            signals = self.find_signals(self.current_arcs, data)

            if signals:
                top_signal = signals[0]
                price = top_signal['price']
                if top_signal['type'] == 'buy':
                    sl = price - self.config.stop_loss_multiplier * top_signal['volatility']
                    tp = price + self.config.profit_ratio * abs(price - sl)
                else:
                    sl = price + self.config.stop_loss_multiplier * top_signal['volatility']
                    tp = price - self.config.profit_ratio * abs(price - sl)

                self.place_order(top_signal['type'], price, sl, tp)

            self.visualize(data)
            time.sleep(self.config.visual_update_seconds)    


if __name__ == "__main__":
    bot = ArcTradingBot()
    bot.run()


In [25]:
import MetaTrader5 as mt5

mt5.initialize()

symbol = "EURUSD.m"
if not mt5.symbol_select(symbol, True):
    print("Failed to select symbol")
    mt5.shutdown()
    quit()

info = mt5.symbol_info(symbol)
tick = mt5.symbol_info_tick(symbol)

print("--- SYMBOL PARAMS ---")
print(f"volume_min    = {info.volume_min}")
print(f"volume_max    = {info.volume_max}")
print(f"volume_step   = {info.volume_step}")
print(f"filling_mode  = {info.filling_mode}")
print(f"trade_mode    = {info.trade_mode}")
print(f"digits        = {info.digits}")
print(f"point         = {info.point}")
print(f"tick_ask      = {tick.ask}")
print(f"tick_bid      = {tick.bid}")

print("\n--- TEST ORDER_CHECK FOR EACH MODE ---")
for mode in [mt5.ORDER_FILLING_FOK, mt5.ORDER_FILLING_IOC, mt5.ORDER_FILLING_RETURN]:
    req = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": info.volume_min,
        "type": mt5.ORDER_TYPE_BUY,
        "price": tick.ask,
        "deviation": 1,
        "magic": 0,
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mode,
    }
    res = mt5.order_check(req)
    print(f"Mode {mode}: retcode={res.retcode}, comment={res.comment}")

mt5.shutdown()


--- SYMBOL PARAMS ---
volume_min    = 0.01
volume_max    = 100.0
volume_step   = 0.01
filling_mode  = 1
trade_mode    = 4
digits        = 5
point         = 1e-05
tick_ask      = 1.13232
tick_bid      = 1.13223

--- TEST ORDER_CHECK FOR EACH MODE ---
Mode 0: retcode=0, comment=Done
Mode 1: retcode=10030, comment=Unsupported filling mode
Mode 2: retcode=10030, comment=Unsupported filling mode


True